In [3]:
import json
import pandas as pd
import numpy as np
from pprint import pprint

In [4]:
def json_parse(lmi_json):
    with open(lmi_json) as json_file:
         json_data = json.load(json_file)
    return json_data

json_dict = json_parse('infile.json')
pprint(json_dict)

{'ads': 1,
 'adsnip': {'click_tracking': 'on'},
 'compactor': 'on',
 'custom': {},
 'partials': {'centerTop_0': {'description': 'this module is used to set up '
                                             'the dfp slots on the page',
                              'instance': 'ad text-align-center labeled_ad',
                              'name': 'ads/dfpslot',
                              'vars': {'dfp_ad_unit': '/51859126/parent_m_ar/gut_890_1',
                                       'dfp_div_id': '300x250_1',
                                       'dfp_height': '250',
                                       'dfp_height_2': '50',
                                       'dfp_height_3': '50',
                                       'dfp_width': '300',
                                       'dfp_width_2': '320',
                                       'dfp_width_3': '300'}},
              'centerTop_1': {'name': 'carousel/ui/title'},
              'centerTop_10': {'description': 'Adds ski

In [14]:
#Set constants

OpenX_ids = {"728": 537088038 , "160": 537088042 , "300":537088040}
Casale_ids = {"728": "154714", "160": "154714", "300": "154714"}
ids = [OpenX_ids, Casale_ids]

#Make sure IDs are strings
for i in ids:
    for key in i:
        i[key] = str(i[key])

type(Casale_ids["728"]) == str

True

# functions

In [6]:
#Mobile Functions
'''Replaces ad units in '''
def replaceDFPUnits_Mobile(json, tag):
    #replaces all ad units with a given tag
    for module in json['partials']:
        name = json['partials'][module]['name']
        if name == 'ads/dfpslot' and json['partials'][module]['vars']['dfp_height'] != '1' :
            json['partials'][module]['vars']['dfp_ad_unit'] = "51859126/parent_m_ar/" + str(tag) + str(determineAdSlot(json['partials'][module]['vars']['dfp_ad_unit']))
        elif name == 'ads/rubicon':
            json['partials'][module]['vars']['rp_kw'] = str(tag) + "_5"
        elif name == 'ads/outbrain':
            json['partials'][module]['vars']['variant'] = str(tag) + "_1"
    return json

#Desktop Functions
def replaceDFPUnits_Desktop(json, tag, casale_ids, openx_ids):
    #replaces all ad units with a given tag
    for module in json['partials']:
        name = json['partials'][module]['name']
        ad_unit = json['partials'][module]['vars']
        varz = json['partials'][module]['vars']
        instance = json['partials'][module]['instance']
        #Replace DFP ad units
        if name == 'ads/dfpslot':
            ad_unit['dfp_ad_unit'] = "51859126/parent_ar/" + "child_ar_" + str(tag[-2:]) + "/" + str(tag) + "_" + str(determineAdSize(varz['dfp_width']))
        elif name == 'ads/outbrain':
            json['partials'][module]['vars']['variant'] = str(tag) + "_1"

def replaceDFPUnits_Desktop2(json, tag, casale_ids, openx_ids):
    #replaces all ad units with a given tag
    for module in json['partials']:
        name = json['partials'][module]['name']
        ad_unit = json['partials'][module]['vars']
        varz = json['partials'][module]['vars']
        #Replace DFP ad units
        if name == 'ads/dfpslot':
            ad_unit['dfp_ad_unit'] = "51859126/parent_ar/" + "child_ar_" + str(tag[-2:]) + "/" + str(tag) + "_" + str(determineAdSize(varz['dfp_width']))
        elif name == 'ads/outbrain':
            json['partials'][module]['vars']['variant'] = str(tag) + "_1"
        elif name == 'links/openx':
            if varz['width'] == '728':
                varz['auid'] = openx_ids["728"]
            elif varz['width'] == '160':
                varz['auid'] = openx_ids["160"]
            elif varz['width'] == '300':
                varz['auid'] = openx_ids["300"]

#Helper functions            
def determineAdSlot(ad_unit):
    if ad_unit[-2:] == "_1":
        end = "_1"
        return end
    elif ad_unit[-2:] == "_2":
        end = "_2"
        return end
    elif ad_unit[-2:] == "_3":
        end = "_3"
        return end
    elif ad_unit[-2:] == "_4":
        end = "_4"
        return end
    
def determineAdSize(ad_unit):
    if ad_unit == '300':
        return "300x250"
    elif ad_unit == '728':
        return "728x90"
    elif ad_unit == '160':
        return "160x600"
    
def replaceDFP_Desktop(module):
    if json['partials'][module]['name'] == 'ads/dfpslot':
        ad_unit['dfp_ad_unit'] = "51859126/parent_ar/" + "child_ar_" + str(tag[-2:]) + "/" + str(tag) + "_" + str(determineAdSize(varz['dfp_width']))
        
def replaceDFP_Mobile(module):
    if module['name'] == 'ads/dfpslot':
        module['dfp_ad_unit'] = "51859126/parent_m_ar/" + str(tag) + str(determineAdSlot(json['partials'][module]['vars']['dfp_ad_unit']))

def replaceOB(module):
    if module['name'] == 'ads/outbrain':
        module['vars']['variant'] = str(tag) + "_1"

def replaceCasale(module, casale_id):
    if module['name'] == 'links/casale':
        module['vars']['casale_id'] = casale_id
        
def replaceOpenX(module, OpenX_ids):
    if module['name'] == 'links/openx':
        module['vars']['auid'] = str(tag) + "_1"
    
#QA Functions   
def QA_div_id():
     for module in json['partials']:
        if json['partials'][module]['name'] == 'ads/dfpslot':
            json['partials'][module]['vars'] = tag
            
   

In [38]:
#new_json = open('json_out','w')
json_data = replaceDFPUnits_Desktop2(json_dict,"ar100",Casale_ids,OpenX_ids)
pprint(json_data)

with open('outfile.json', 'w') as outfile:
    json.dump(json_data, outfile)

KeyError: 'vars'

In [12]:
#new_json = open('json_out','w')
json_data = replaceDFPUnits_Mobile(json_dict,"gut_742")
pprint(json_data)

with open('outfile.json', 'w') as outfile:
    json.dump(json_data, outfile)

{'ads': 1,
 'adsnip': {'click_tracking': 'on'},
 'compactor': 'on',
 'custom': {},
 'partials': {'centerTop_0': {'description': 'this module is used to set up '
                                             'the dfp slots on the page',
                              'instance': 'ad text-align-center labeled_ad',
                              'name': 'ads/dfpslot',
                              'vars': {'dfp_ad_unit': '51859126/parent_m_ar/gut_742_1',
                                       'dfp_div_id': '300x250_1',
                                       'dfp_height': '250',
                                       'dfp_height_2': '50',
                                       'dfp_height_3': '50',
                                       'dfp_width': '300',
                                       'dfp_width_2': '320',
                                       'dfp_width_3': '300'}},
              'centerTop_1': {'name': 'carousel/ui/title'},
              'centerTop_10': {'description': 'Adds skin